In [1]:
import pandas as pd

In [38]:
old_df = pd.read_csv("C:/Users/ki87ujmn/Downloads/merged_file.csv")
mapping = old_df.set_index('site_id')['weir_length']


In [39]:
raw_incidents = pd.read_excel(io='../data/Master Copy of LHD Fatalities Database.xlsx',
                       sheet_name='Incidents')
# let's see how many fatalities we have in the database...
print(f'There are {len(raw_incidents)} fatalities in the fatalities database.')

There are 600 fatalities in the fatalities database.


In [40]:
raw_incidents['weir_length'] = raw_incidents['site_id'].map(mapping)

In [41]:
# now let's see how many have complete dates
date_cols = ["day", "month", "year"]
incident_df = raw_incidents[(raw_incidents[date_cols] != 0).all(axis=1)].copy()
incident_df["date"] = pd.to_datetime(incident_df[["year", "month", "day"]])
incident_df["date"] = incident_df["date"].dt.strftime("%Y-%m-%d")
print(f'There are {len(incident_df)} fatalities with complete dates.')

There are 550 fatalities with complete dates.


In [42]:
# let's look at the locations now...
site_df = pd.read_excel(io='../data/Master Copy of LHD Fatalities Database.xlsx',
                       sheet_name='Sites')
# let's see how many dams we have in the database...
print(f'There are {len(site_df)} fatalities in the fatalities database.')

There are 356 fatalities in the fatalities database.


In [58]:
# long_df = site_df.merge(incident_df[["site_id", "date"]], on="site_id", how="left")
# long_df = long_df.dropna(subset=["date"])
long_df = pd.read_excel('../data/LHD Sites and Fatalities.xlsx')

print(f"There are {len(long_df)} fatalities with complete dates.\n"
      f"There are {long_df['site_id'].nunique()} sites with complete dates.")
long_df = long_df[long_df["comments"] != "removed"]
print(f"There are {len(long_df)} fatalities at current low-head dams.\n"
      f"There are {long_df['site_id'].nunique()} current low-head dams.")
# long_df.to_excel(excel_writer='../data/LHD Sites and Fatalities.xlsx',
#                  sheet_name='Fatalities', index=False)

There are 550 fatalities with complete dates.
There are 344 sites with complete dates.
There are 449 fatalities at current low-head dams.
There are 284 current low-head dams.


In [59]:
# now let's see how many we can test with the NWM
start = "1979-02-01"
end   = "2023-01-31"
nwm_df = long_df[(long_df["date"] >= start) & (long_df["date"] <= end)]
print(f"There are {len(nwm_df)} fatalities with NWM-compatible dates.\n"
      f"There are {nwm_df['site_id'].nunique()} sites with NWM-compatible dates.")
nwm_df.to_excel(excel_writer='../data/NWM LHD Sites and Fatalities.xlsx',
                 sheet_name='Fatalities', index=False)

There are 372 fatalities with NWM-compatible dates.
There are 264 sites with NWM-compatible dates.


In [60]:
# now let's see how many we can test with the NWM
geo_df = long_df[long_df["date"] >= "1940-01-01"]
print(f"There are {len(geo_df)} fatalities with GEOGLOWS-compatible dates.\n"
      f"There are {geo_df['site_id'].nunique()} sites with GEOGLOWS-compatible dates.")
geo_df.to_excel(excel_writer='../data/GEOGLOWS LHD Sites and Fatalities.xlsx',
                 sheet_name='Fatalities', index=False)

There are 434 fatalities with GEOGLOWS-compatible dates.
There are 281 sites with GEOGLOWS-compatible dates.


In [2]:
import pandas as pd

# 1. Load the excel file
file_path = '/Users/kennyquintana/Developer/lhd-processor/lhd_processor/data/LHD Sites and Fatalities copy.xlsx'
df = pd.read_excel(file_path)

# 2. Define the columns for each group
site_columns = [
    'site_id', 'name', 'latitude', 'longitude',
    'city', 'county', 'state', 'weir_length', 'comments'
]

incident_columns = [
    'site_id', 'date'
]

# 3. Create the Site-Specific DataFrame
# We drop duplicates on 'site_id' so each dam is listed only once
sites_df = df[site_columns].drop_duplicates(subset='site_id').reset_index(drop=True)

# 4. Create the Incident-Specific DataFrame
# We keep all rows here, as multiple incidents can occur at one site_id
incidents_df = df[incident_columns].reset_index(drop=True)

# 5. Export to an Excel file with two sheets
output_filename = '/Users/kennyquintana/Developer/lhd-processor/lhd_processor/data/LHD_Split_Data.xlsx'

with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
    sites_df.to_excel(writer, sheet_name='Sites', index=False)
    incidents_df.to_excel(writer, sheet_name='Incidents', index=False)

print(f"Successfully created {output_filename} with two sheets.")
print(f"Unique Sites: {len(sites_df)}")
print(f"Total Incidents: {len(incidents_df)}")

Successfully created /Users/kennyquintana/Developer/lhd-processor/lhd_processor/data/LHD_Split_Data.xlsx with two sheets.
Unique Sites: 344
Total Incidents: 550


In [1]:
import pandas as pd

# --- Configuration: Set your Date Ranges Here ---
# Adjust these based on the specific Reanalysis version you are using.
# NWM Retrospective 2.1 is typically 1979-02-01 to 2020-12-31.
# NWM 3.0 extends further.
NWM_START_DATE = '1979-02-01'
NWM_END_DATE = '2023-01-31'

# GEOGLOWS v2 typically goes back to 1940.
GEOGLOWS_START_DATE = '1940-01-01'

def process_and_split_data(input_path):
    print(f"Reading: {input_path}")
    xls = pd.ExcelFile(input_path)

    # 1. Load Data
    df_sites = pd.read_excel(xls, 'Sites')
    df_incidents = pd.read_excel(xls, 'Incidents')

    # 2. Universal Cleaning: Remove rows where comments == 'removed'
    # Check 'sites'
    if 'comments' in df_sites.columns:
        df_sites = df_sites[df_sites['comments'].astype(str).str.lower().str.strip() != 'removed']

    # Check 'incidents'
    if 'comments' in df_incidents.columns:
        df_incidents = df_incidents[df_incidents['comments'].astype(str).str.lower().str.strip() != 'removed']

    # 3. Date Filtering Preparation
    # Ensure the date column is actually datetime objects.
    date_col = 'date'
    df_incidents[date_col] = pd.to_datetime(df_incidents[date_col], errors='coerce')

    # Drop rows where date failed to parse (NaT) if necessary
    df_incidents = df_incidents.dropna(subset=[date_col])

    # 4. Create NWM Slice
    mask_nwm = (df_incidents[date_col] >= NWM_START_DATE) & (df_incidents[date_col] <= NWM_END_DATE)
    df_incidents_nwm = df_incidents.loc[mask_nwm]

    # 5. Create GEOGLOWS Slice
    mask_geoglows = df_incidents[date_col] >= GEOGLOWS_START_DATE
    df_incidents_geoglows = df_incidents.loc[mask_geoglows]

    print(f"Total Incidents (Cleaned): {len(df_incidents)}")
    print(f"NWM Compatible Incidents: {len(df_incidents_nwm)}")
    print(f"GEOGLOWS Compatible Incidents: {len(df_incidents_geoglows)}")

    # 6. Save NWM File (Sites + Filtered Incidents)
    with pd.ExcelWriter('/Volumes/KenDrive/LHD_Project/nwm_data.xlsx', engine='openpyxl') as writer:
        df_sites.to_excel(writer, sheet_name='Sites', index=False)
        df_incidents_nwm.to_excel(writer, sheet_name='Incidents', index=False)

    # 7. Save GEOGLOWS File (Sites + Filtered Incidents)
    with pd.ExcelWriter('/Volumes/KenDrive/LHD_Project/geoglows_data.xlsx', engine='openpyxl') as writer:
        df_sites.to_excel(writer, sheet_name='Sites', index=False)
        df_incidents_geoglows.to_excel(writer, sheet_name='Incidents', index=False)

    print("\nSuccess! Created 'nwm_data.xlsx' and 'geoglows_data.xlsx'.")

# Run the function
# Replace 'master_data.xlsx' with your actual file name
process_and_split_data('/Volumes/KenDrive/LHD_Project/LHD_Database_Full.xlsx')

Reading: /Volumes/KenDrive/LHD_Project/LHD_Database_Full.xlsx
Total Incidents (Cleaned): 550
NWM Compatible Incidents: 460
GEOGLOWS Compatible Incidents: 534

Success! Created 'nwm_data.xlsx' and 'geoglows_data.xlsx'.


In [5]:
import pandas as pd

# 1. Read the entire Excel file. sheet_name=None reads all sheets into a dictionary.
input_file = '/Volumes/KenDrive/LHD_Project/nwm_data.xlsx'
all_sheets = pd.read_excel(input_file, sheet_name=None)

# 2. Extract the relevant DataFrames
sites_df = all_sheets['Sites']
incidents_df = all_sheets['Incidents']

# 3. Get the valid site_ids from the 'Sites' tab
valid_site_ids = sites_df['site_id'].unique()

# 4. Filter the 'Incidents' tab
# Keep only rows where 'site_id' is in the list of valid_site_ids
filtered_incidents_df = incidents_df[incidents_df['site_id'].isin(valid_site_ids)]

# 5. Update the dictionary with the filtered dataframe
all_sheets['Incidents'] = filtered_incidents_df

# 6. Write all sheets (including the unchanged ones) to a new Excel file
output_file = '/Volumes/KenDrive/LHD_Project/nwm_data.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in all_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Successfully saved filtered data to {output_file}")

Successfully saved filtered data to /Volumes/KenDrive/LHD_Project/nwm_data.xlsx


In [7]:
import pandas as pd

# 1. Read the entire Excel file.
input_file = '/Volumes/KenDrive/LHD_Project/geoglows_data.xlsx'
all_sheets = pd.read_excel(input_file, sheet_name=None)

# 2. Extract the relevant DataFrames
sites_df = all_sheets['Sites']
incidents_df = all_sheets['Incidents']

# 3. Get the valid site_ids from the 'Incidents' tab (The Source)
# This identifies which sites actually have recorded incidents
incident_site_ids = incidents_df['site_id'].unique()

# 4. Filter the 'Sites' tab (The Target)
# Keep only rows in 'Sites' where the 'site_id' appears in the Incidents list
filtered_sites_df = sites_df[sites_df['site_id'].isin(incident_site_ids)]

# 5. Update the dictionary with the filtered dataframe
all_sheets['Sites'] = filtered_sites_df

# 6. Write all sheets to the Excel file
output_file = '/Volumes/KenDrive/LHD_Project/geoglows_data_test.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, df in all_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Successfully filtered 'Sites' based on 'Incidents' and saved to {output_file}")
print(f"Original Sites count: {len(sites_df)}")
print(f"Filtered Sites count: {len(filtered_sites_df)}")

Successfully filtered 'Sites' based on 'Incidents' and saved to /Volumes/KenDrive/LHD_Project/geoglows_data_test.xlsx
Original Sites count: 284
Filtered Sites count: 281
